In [11]:
import sklearn
import corus
import pandas as pd
import numpy as np
import nltk
import json
import tqdm

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import string
import re

from pprint import pprint

from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

from pymystem3 import Mystem

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\firem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\firem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
MAX_LENTA_DATASET_ROWS = 10000
RANDOM_STATE = 5555
NUM_TOPICS = 10
CHUNK_SIZE = 750

BIGARTM_SEED = 2023

#### Загрузить набор данных Lenta.ru с помощью пакета Corus

In [13]:
!pip install requests

In [14]:
import requests
import os

files = os.listdir()


def download_file(url, name):
    response = requests.get(url)
    open(name, "wb").write(response.content)

In [15]:
import requests
from corus import load_lenta2

url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2"

path = 'lenta-ru-news.csv.bz2'

download_file(url, path)
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [25]:
wnl = WordNetLemmatizer()
STOP_WORDS = stopwords.words('russian')

download_file(
    "https://raw.githubusercontent.com/danilkladnitsky/nlp-tasks/hw_5/hw_5/stopwords-ru.json", "stopwords-ru.json")
download_file(
    "https://raw.githubusercontent.com/danilkladnitsky/nlp-tasks/hw_5/hw_5/stopwords-ru.txt", "stopwords-ru.txt")


external_stop_word_1 = open(
    "stopwords-ru.txt", "r", encoding="utf-8").read().replace('\n', ' ').split(" ")

raw_json = open(
    "stopwords-ru.json", "r", encoding="utf-8").read()
external_stop_word_2 = json.loads(raw_json)

external_stop_words = set(external_stop_word_1 + external_stop_word_2)

STOP_WORDS.extend(external_stop_words)

m = Mystem()


def lemmatize_text(token): return m.lemmatize(token)


def normalize_text(text):
    normalized_text = re.sub('\w*\d\w*', '', text).translate(str.maketrans(
        '', '', string.punctuation)).lower()

    lemmatized_text = lemmatize_text(normalized_text)

    ready_tokens = [word for word in lemmatized_text if (word).strip()]
    result = ' '.join([word
                      for word in ready_tokens if word not in STOP_WORDS])

    return result

<>:28: DeprecationWarning: invalid escape sequence \w
<>:28: DeprecationWarning: invalid escape sequence \w
C:\Users\firem\AppData\Local\Temp\ipykernel_9812\819238068.py:28: DeprecationWarning: invalid escape sequence \w
  normalized_text = re.sub('\w*\d\w*', '', text).translate(str.maketrans(


'мама мыть рама'

In [16]:
def load_lenta_to_list(path, max_number=None):
    records = load_lenta2(path)
    texts, titles, dates, original_topics, raw_content = [], [], [], [], []
    for i, record in enumerate(records):
        texts.append(normalize_text(record.text))
        titles.append(record.title)
        dates.append(record.date)
        original_topics.append(record.topic)
        raw_content.append(record.text)

        if not max_number is None:
            if i >= max_number-1:
                break
    result = {}
    result['topic'] = titles
    result['date'] = dates
    result['original_topics'] = original_topics
    result['content'] = texts
    result['raw_content'] = raw_content

    return result

In [26]:
lenta_list = load_lenta_to_list(path, max_number=MAX_LENTA_DATASET_ROWS)

lentaFrame = pd.DataFrame(lenta_list)
lentaFrame

#### Обучить LDA модель, постараться подобрать адекватные параметры (num_topics, passes, alpha, iterations…)

In [ ]:
tokenized_content = lentaFrame.content.apply(lambda x: x.split())
id2word = corpora.Dictionary(tokenized_content)

In [ ]:
corpus = [id2word.doc2bow(text) for text in tokenized_content]
corpus_example = [[(id2word[id], freq) for id, freq in cp]
                  for cp in corpus[:2]]
corpus_example[0][:50]

[('«русский', 1),
 ('австрийцев', 1),
 ('автомобили', 1),
 ('артиллерийском', 1),
 ('артиллерийскую', 1),
 ('артиллерию', 1),
 ('батареей', 1),
 ('бежала', 1),
 ('безуспешными', 1),
 ('ближе', 1),
 ('бои', 1),
 ('большая', 1),
 ('борьбу', 1),
 ('бою', 1),
 ('бросив', 1),
 ('венгрии', 1),
 ('взяли', 1),
 ('вступили', 1),
 ('вылазки', 1),
 ('высот', 1),
 ('галиции', 1),
 ('гарнизона', 1),
 ('германской', 1),
 ('германцев', 1),
 ('дембицу', 1),
 ('достиг', 1),
 ('друскеник', 1),
 ('закончились', 1),
 ('заняли', 1),
 ('захватываются', 1),
 ('значительного', 1),
 ('инвалид»', 1),
 ('калибры', 1),
 ('колонна', 1),
 ('крепости', 1),
 ('крепостью', 1),
 ('материальная', 1),
 ('напряжения', 1),
 ('начал', 1),
 ('нашей', 1),
 ('неприятель', 1),
 ('неприятельский', 1),
 ('новые', 1),
 ('обнаруживается', 1),
 ('обоз', 1),
 ('обстреливалась', 1),
 ('огонь', 1),
 ('орудия', 1),
 ('осовцу', 1),
 ('остаются', 1)]

In [ ]:
def get_coherence(corpus, topics, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           random_state=RANDOM_STATE,
                                           chunksize=CHUNK_SIZE,
                                           num_topics=topics,
                                           alpha=a,
                                           eta=b,
                                           workers=3)

    coherence_model_lda = CoherenceModel(
        model=lda_model, texts=tokenized_content, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [ ]:
%%time

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=NUM_TOPICS,
                                       random_state=RANDOM_STATE,
                                       chunksize=CHUNK_SIZE,
                                       alpha='symmetric',
                                       per_word_topics=True,
                                       workers=3)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"россии" + 0.004*"сообщает" + 0.004*"словам" + 0.003*"чечне" + '
  '0.003*"новости" + 0.003*"боевиков" + 0.003*"риа" + 0.002*"заявил" + '
  '0.002*"президента" + 0.002*"сообщил"'),
 (1,
  '0.007*"россии" + 0.004*"заявил" + 0.004*"сообщает" + 0.004*"словам" + '
  '0.003*"президента" + 0.003*"сообщил" + 0.002*"рф" + 0.002*"новости" + '
  '0.002*"чечни" + 0.002*"отметил"'),
 (2,
  '0.006*"россии" + 0.005*"новости" + 0.005*"риа" + 0.004*"сообщили" + '
  '0.003*"сообщает" + 0.003*"мвд" + 0.003*"боевиков" + 0.003*"словам" + '
  '0.003*"районе" + 0.003*"данным"'),
 (3,
  '0.004*"сообщает" + 0.003*"рф" + 0.003*"россии" + 0.003*"новости" + '
  '0.003*"риа" + 0.003*"словам" + 0.003*"рублей" + 0.003*"заявил" + '
  '0.002*"президента" + 0.002*"сша"'),
 (4,
  '0.005*"сообщает" + 0.003*"россии" + 0.003*"компании" + 0.003*"словам" + '
  '0.003*"долларов" + 0.003*"риа" + 0.002*"заявил" + 0.002*"чемпионата" + '
  '0.002*"новости" + 0.002*"банка"'),
 (5,
  '0.006*"россии" + 0.005*"риа" + 0

#### Визуализировать результаты работы LDA с помощью pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.007139  0.012709       1        1  17.863108
5      0.037084  0.016625       2        1  13.221544
7     -0.008964  0.011407       3        1  11.519166
2      0.070591 -0.029672       4        1  11.109659
6     -0.024430 -0.014380       5        1   9.024520
9     -0.003150  0.001717       6        1   8.774744
1     -0.007493  0.041453       7        1   8.574785
4     -0.036464 -0.054066       8        1   7.886109
3     -0.033793  0.011172       9        1   7.877441
0      0.013759  0.003035      10        1   4.148923, topic_info=               Term         Freq        Total Category  logprob  loglift
471          россии  5222.000000  5222.000000  Default  30.0000  30.0000
1506          чечне  1319.000000  1319.000000  Default  29.0000  29.0000
1369       боевиков  1097.000000  1097.000000  Default  28.0000  28.0000
4548         рублей  1161.000000  1161.000000  Default  27.0000  27.0000
998        сообщили  1380.000000  1380.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
608        долларов    59.093413  2062.548678  Topic10  -6.5650  -0.3703
2061            сша    59.869696  2295.324721  Topic10  -6.5520  -0.4641
1709       компании    51.405130  1333.060112  Topic10  -6.7044  -0.0732
521          данным    52.332049  1607.069763  Topic10  -6.6865  -0.2422
652   правительства    50.243400  1227.389322  Topic10  -6.7273  -0.0134

[915 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
7127       5  0.922989      atlantis
29203      6  0.922281            bt
61450      1  0.048135          card
61450      2  0.770156          card
61450      3  0.096269          card
...      ...       ...           ...
36101      6  0.016907  ястржембский
36101      7  0.231062  ястржембский
36101      8  0.022543  ястржембский
36101      9  0.033814  ястржембский
36101     10  0.180341  ястржембский

[3242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 8, 3, 7, 10, 2, 5, 4, 1])

#### Посчитать внутренние метрики обученных моделей LDA (с разными параметрами) и сравнить, соответствует ли метрика визуальному качеству работы моделей

In [ ]:
get_coherence(corpus, NUM_TOPICS, 1, 1)

0.35703786165147566

In [ ]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 3
max_topics = 12
step_size = 3
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
               corpus]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                 }

if 1 == 0:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha) *
                     len(topics_range)))

    for k in topics_range:
        for a in alpha:
            for b in beta:
                cv = get_coherence(corpus=corpus,
                                   topics=k, a=a, b=b)

                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)

                pbar.update(1)

    pd.DataFrame(model_results).to_csv(
        './lda_tuning_results.csv', index=False)
    pbar.close()

#### Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик

In [ ]:
!pip install hyperopt

In [ ]:
import artm
from sklearn.feature_extraction.text import CountVectorizer
import os
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"  # protobuf fix

/home/partnadem/.pyenv/versions/3.9.12/envs/wsl-3.9.12/lib/python3.9/site-packages/future/standard_library/__init__.py:65: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [ ]:
tokenized_content = lentaFrame.content.to_list()

cv = CountVectorizer(max_features=1000, max_df=0.8,
                     min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform(tokenized_content).todense()).T

print(n_wd.shape)

(1000, 10000)


In [ ]:
token_list = [i for i in cv.vocabulary_.keys()]
bv = artm.BatchVectorizer(data_format='bow_n_wd',
                          n_wd=n_wd, vocabulary=token_list)

/home/partnadem/.pyenv/versions/3.9.12/envs/wsl-3.9.12/lib/python3.9/site-packages/artm/batches_utils.py:227: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [ ]:
def get_artm_model(seed=BIGARTM_SEED):
    model = artm.ARTM(num_topics=40, dictionary=bv.dictionary,
                      cache_theta=True, seed=seed)
    model.scores.add(artm.PerplexityScore(
        name='perplexity_score', dictionary=bv.dictionary))

    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(
        name='top_tokens_score', num_tokens=100))

    model.fit_offline(bv, num_collection_passes=30)
    return model

In [ ]:
%%time

model = get_artm_model()
model.score_tracker["perplexity_score"].last_value

CPU times: user 35.8 s, sys: 1.69 s, total: 37.5 s
Wall time: 8.3 s


239.1787872314453

In [ ]:
def get_model_instance(context):
    model = artm.ARTM(
        num_topics=context['num_topics'], dictionary=bv.dictionary, cache_theta=True)

    model.scores.add(artm.PerplexityScore(
        name='perplexity_score', dictionary=bv.dictionary))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(
        name='top_tokens_score', num_tokens=10))

    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(
            name='SparsePhi',
            tau=context['phi_tau']
        ),
    )
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(
            name='SparseTheta',
            tau=context['theta_tau']
        ),
    )
    model.regularizers.add(
        artm.DecorrelatorPhiRegularizer(
            name='DecorrelatorPhi',
            tau=context['decorrelation_tau']
        ),
    )

    model.fit_offline(bv, num_collection_passes=30)

    perplexity_score = model.score_tracker["perplexity_score"].last_value
    print("Perplexity:", perplexity_score)
    return {'loss': perplexity_score, 'status': STATUS_OK}

In [ ]:
context = {
    'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
    'theta_tau': hp.uniform('SparseTheta', -1, 1),
    'phi_tau': hp.uniform('SparsePhi', -1, 1),
    'num_topics': hp.choice('num_topics', [3, 6, 9, 12, 15]),
}

trials = Trials()

result = fmin(
    fn=get_model_instance,
    space=context,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials,
)

print(result)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

Perplexity:                                           
419.776611328125                                      
Perplexity:                                                                   
547.4457397460938                                                             
Perplexity:                                                                   
348.8782958984375                                                             
Perplexity:                                                                    
487.32366943359375                                                             
Perplexity:                                                                    
403.5289611816406                                                              
Perplexity:                                                                    
380.49285888671875                                                             
Perplexity:                                                                    
452.0794067382

#### Winner
{'DecorrelatorPhi': 3106.636084605295, 'SparsePhi': 0.44741088069669166, 'SparseTheta': 0.005391870879176919, 'num_topics': 15}

In [ ]:
best_settings = {'DecorrelatorPhi': 3106.636084605295, 'SparsePhi': 0.44741088069669166,
                 'SparseTheta': 0.005391870879176919, 'num_topics': 4}

model = artm.ARTM(num_topics=15,
                  dictionary=bv.dictionary, cache_theta=True, seed=BIGARTM_SEED)
model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                      dictionary=bv.dictionary))

model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))
model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))

model.regularizers.add(artm.DecorrelatorPhiRegularizer(
    name='DecorrelatorPhi', tau=best_settings['DecorrelatorPhi']))
model.regularizers.add(artm.SmoothSparsePhiRegularizer(
    name='SparsePhi', tau=best_settings['SparsePhi']))
model.regularizers.add(artm.SmoothSparseThetaRegularizer(
    name='SparseTheta', tau=best_settings['SparseTheta']))


model.fit_offline(bv, num_collection_passes=30)


model.score_tracker["perplexity_score"].last_value

In [ ]:
def prepare_vis_data():
    phi = model.get_phi()
    theta = model.get_theta().to_numpy().T
    theta = theta / theta.sum(axis=1, keepdims=1)
    data = {'topic_term_dists': phi.to_numpy().T,
            'doc_topic_dists': theta,
            'doc_lengths': n_wd.sum(axis=0).tolist(),
            'vocab': phi.T.columns,
            'term_frequency': n_wd.sum(axis=1).tolist()}
    return data


model_data = prepare_vis_data()
model_vis = pyLDAvis.prepare(**model_data)

pyLDAvis.save_html(model_vis, 'visual.html')
pyLDAvis.display(model_vis)

SyntaxError: invalid syntax (892953566.py, line 18)